##  07.7 - Learning Policy on Image + Attributes

Fully connected network with:
- Input: 
    - **184** size vector with:
        * client information: [current latitude, current longitud, destination latitude, destination longitude]
        * vehicles information: [load, queue, current latitude, current longitud, next latitude, next longitude]
    - Image with 31 channel: 
        * Channel 0 --> client representation. its current location is marked as 1; its destination as -1
        * Channels > 0 --> vehicle current location (1)
        
- Output: One hot encoding vector with 30 entries representing the available vehicles


The attribute input is merged later with a partial output of the network

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import pandas as pd
import random
import tqdm
import time

import matplotlib.pyplot as plt

from torch.utils.data import Dataset
from torch import optim

In [2]:
class ImagesP2(nn.Module):
    def __init__(self, inp, im_size):
        super().__init__()

        ins = 10
        self.cs = 5
        self.im_size = im_size
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(inp, ins, kernel_size=self.cs),
            nn.BatchNorm2d(ins),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(ins, 5, kernel_size=self.cs),
            nn.BatchNorm2d(5),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(5, 2, kernel_size=self.cs),
            nn.BatchNorm2d(2),
            nn.ReLU(),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(2*(im_size - 3*(self.cs-1))*(im_size - 3*(self.cs-1)), 264),
            nn.ReLU(),
        
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(264, inp-1),
        )
        
        

    def forward(self, x):
                
        
        x0 = x
        x1 = self.conv1(x0)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        x4 = x3.view(-1,2*(self.im_size - 3*(self.cs-1))*(self.im_size - 3*(self.cs-1)))
        x5 = self.fc1(x4)
        x6 = self.fc2(x5)
        return x6

In [20]:
def epoch_train(model, train_table, test_tables, optimizer, criterion, e, im_size=30, clipped=False):
    
    sum_loss = 0
    acc = []
    idx_failures = []
    dist = 1/(im_size-1)

    # train data
    for k, TABLE in enumerate(train_tables):
        
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()

        idx = list(data.keys())
        nv = len(data[idx[0]]) - 1
        
        

        for b in range(0, len(idx), mini_batch_size):
            
            # idx of clients to analyse
            t_idx = idx[b:b+mini_batch_size]
            
            if clipped:
                x = np.zeros((len(t_idx), 2, im_size, im_size))
            else:
                x = np.zeros((len(t_idx), nv+1, im_size, im_size))

            
            # iterate along clients
            for k, cl in enumerate(t_idx):
                
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1
                
                if clipped:
                    x[k][1][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
                # iterate along vehicles
                else:
                    for i in range(1,31):
                        x[k][i][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1

            y = np.hstack([data_y[i] for i in t_idx])

            train_x = torch.tensor(x).type(torch.FloatTensor)
            train_y = torch.tensor(y).type(torch.LongTensor)
            
            # set gradient to zero
            optimizer.zero_grad()
            
            # compute output
            # output1, output2 = model(train_x, train_x_aux)
            # batch_loss = criterion(output1, train_y) + criterion(output2, train_y)
            output1= model(train_x)
            batch_loss = criterion(output1, train_y)
            
            
            batch_loss.backward()
            optimizer.step()

            sum_loss = sum_loss + batch_loss.item()
            _, a = torch.max(output1,1)
            acc.append(float((train_y == a).sum())/len(train_y))
            
            
            #acc.append(100*output.detach().max(1)[1].eq(train_y).sum().item()/(train_y.shape[0]*train_y.shape[1]*train_y.shape[2]))

    
    test_loss = 0
    test_acc = []
    
    for k,TABLE in enumerate(test_tables):
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        
        idx = list(data.keys())
        #random.shuffle(idx)
        
        for b in range(0, len(idx), mini_batch_size):

            t_idx = idx[b:b+mini_batch_size]
            
            if clipped:
                x = np.zeros((len(t_idx), 2, im_size, im_size))
            else:
                x = np.zeros((len(t_idx), nv+1, im_size, im_size))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1
                if clipped:
                    x[k][1][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
                else:
                    for i in range(1,31):
                        x[k][i][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
            
            y = np.hstack([data_y[i] for i in t_idx])
            
            test_x = torch.tensor(x).type(torch.FloatTensor)
            
            test_y = torch.tensor(y).type(torch.LongTensor)
            
            output1= model(test_x)
            batch_loss = criterion(output1, test_y) 
            

            test_loss += batch_loss.item()
            _, a = torch.max(output1,1)
            test_acc.append(float((test_y == a).sum())/len(test_y))
            idx_failures += [t_idx[i] for i in np.where(test_y != a)[0]]
            
            
    print('\rEpoch {}. Train Loss: {:.3f} Accuracy: {:.3f} Test Loss: {:.3f} Accuracy: {:.3f}'.format(e+1, sum_loss, np.mean(acc), test_loss,np.mean(test_acc)), end="")
    #print('Epoch {}. Test Loss: {:.3f}'.format(e+1,test_loss))
    return sum_loss, np.sum(acc)/len(acc), test_loss, np.sum(test_acc)/len(test_acc), idx_failures
    #return sum_loss, np.mean(acc)/len(acc)

### General configurations

In [7]:
random.seed(4915)
global MAX_V, mini_batch_size, n_epochs, lr, inp_size
MAX_V = 30
mini_batch_size = 50
n_epochs=30

tables = list(range(1,90))
random.shuffle(tables)

lr = 0.0001
inp_size = 4 + MAX_V*6

train_tables, test_tables, validation_tables = \
tables[:int(len(tables)*0.6)], tables[int(len(tables)*0.6):int(len(tables)*0.85)], tables[int(len(tables)*0.85):]

### Functions

In [12]:
def evaluating_model(im_size, model, n_epochs):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss() 
    
    loss, acc, test_loss, test_acc, idx_f, times = [], [], [], [], [], []

    for epoch in range(n_epochs):
        current_t = time.time()
        train_l, accuracy, test_l, test_a, idx_failures = \
        epoch_train(model, train_tables, test_tables, optimizer, criterion, epoch, im_size=im_size)
        
        times.append(time.time() - current_t)
        loss.append(train_l)
        test_loss.append(test_l)
        acc.append(accuracy)
        test_acc.append(test_a)
        idx_f.append(idx_failures)

    print('\nAverage time per epoch {:.3f}s +- {:.3f}'.format(np.mean(times), 2*np.std(times)))

    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc ==  max_acc)

    print('Max accuracy of {:.3f} achieved at epoch {}'.format(max_acc, iter_max[0][0]))
    
    return loss, acc, test_loss, test_acc, idx_f, times


def visualize_results(loss, test_loss, acc, test_acc, idx_f):
    f, ax = plt.subplots(2, 1, figsize=(20,8))
    ax[0].plot(loss)
    ax[0].plot(test_loss)
    ax[1].plot(acc)
    ax[1].plot(test_acc)
    plt.show()
    
    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc ==  max_acc) 
    iter_max = iter_max[0][0]

    fig, ax = plt.subplots(1,5, figsize=(20,4))
    k = 0
    idx = idx_f[:iter_max+1]
    for j, i in enumerate(idx[::-1]):
        if len(i) > 0:
            ax[k].hist(i, bins=50)
            ax[k].set_xlabel('Client')
            ax[k].set_ylabel('Num errors')
            ax[k].set_title('Iteration {}\nwith acc:{:.3f}'.format(-j + iter_max, test_acc[-j + iter_max]))
            k += 1
        if k == 5:
            plt.show()
            break

### Model with image size = 30x30

In [9]:
for i in range(5):
    loss, acc, test_loss, test_acc, idx_f, times= evaluating_model(30)
#visualize_results(loss, test_loss, acc, test_acc, idx_f)

Epoch 30. Train Loss: 133.113 Accuracy: 0.635 Test Loss: 138.328 Accuracy: 0.136
Average time per epoch 12.610s +- 0.951
Max accuracy of 0.146 achieved at epoch 11
Epoch 30. Train Loss: 131.406 Accuracy: 0.637 Test Loss: 141.776 Accuracy: 0.127
Average time per epoch 12.748s +- 0.544
Max accuracy of 0.149 achieved at epoch 11
Epoch 30. Train Loss: 135.746 Accuracy: 0.623 Test Loss: 137.824 Accuracy: 0.138
Average time per epoch 12.495s +- 0.490
Max accuracy of 0.149 achieved at epoch 7
Epoch 30. Train Loss: 131.459 Accuracy: 0.641 Test Loss: 142.067 Accuracy: 0.137
Average time per epoch 12.562s +- 0.322
Max accuracy of 0.145 achieved at epoch 12
Epoch 30. Train Loss: 131.940 Accuracy: 0.642 Test Loss: 141.049 Accuracy: 0.137
Average time per epoch 12.428s +- 0.417
Max accuracy of 0.146 achieved at epoch 13


In [14]:
im_size = 30
model30 = ImagesP2(31, im_size)
loss, acc, test_loss, test_acc, idx_f, times= evaluating_model(30, model30, 11)

Epoch 11. Train Loss: 230.711 Accuracy: 0.319 Test Loss: 114.935 Accuracy: 0.138
Average time per epoch 11.637s +- 0.424
Max accuracy of 0.138 achieved at epoch 10


In [15]:
torch.save(model30, 'model_simple_30.pt')

/Users/nataliebolon/miniconda3/envs/vehicles/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ImagesP2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/nataliebolon/miniconda3/envs/vehicles/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/nataliebolon/miniconda3/envs/vehicles/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/nataliebolon/miniconda3/envs/vehicles/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type 

In [21]:
im_size = 30
model30_clipped = ImagesP2(2, im_size)
loss, acc, test_loss, test_acc, idx_f, times= evaluating_model(30, model30, 11)
torch.save(model30_clipped, 'model_simple_30_clipped.pt')

Epoch 11. Train Loss: 176.716 Accuracy: 0.487 Test Loss: 124.270 Accuracy: 0.141
Average time per epoch 10.905s +- 0.660
Max accuracy of 0.142 achieved at epoch 5


In [22]:
torch.save(model50, 'model_simple_30_clipped.pt')